In [4]:
import numpy as np
import random
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing import text
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Model
import tensorflow as tf
import logging

tf.get_logger().setLevel(logging.ERROR)

In [6]:
EPOCHS = 20
BATCH_SIZE = 128
MAX_WORDS = 10000
READ_LINTS = 60000
LAYER_SIZE = 256
EMBEDDING_WIDTH = 128
TEST_PERCENT = 0.2
SAMPLE_SIZE = 20
OOV_WORD = "UNK"
PAD_INDEX = 0
OOV_INDEX = 1
START_INDEX = MAX_WORDS - 2
STOP_INDEX = MAX_WORDS - 1
MAX_LENGTH = 60
FILE_NAME = r"D:\it\dataset\fra.txt"

In [7]:
def read_file_combined(file_name, max_len):
    file = open(file_name, 'r', encoding='UTF-8')
    src_word_sequences = []
    dest_word_sequences = []
    for i, line in enumerate(file):
        if i == READ_LINTS:
            break
        pair = line.split('\t')
        word_sequence = text.text_to_word_sequence(pair[1])
        src_word_sequence = word_sequence[0:max_len]
        src_word_sequences.append(src_word_sequence)
        word_sequence = text.text_to_word_sequence(pair[0])
        dest_word_sequence = word_sequence[0:max_len]
        dest_word_sequences.append(dest_word_sequence)
    file.close()
    return src_word_sequences, dest_word_sequences

In [8]:
def tokenize(sequences):
    tokenizer = text.Tokenizer(num_words=MAX_WORDS - 2, oov_token=OOV_WORD)
    tokenizer.fit_on_texts(sequences)
    token_sequences = tokenizer.texts_to_sequences(sequences)
    return tokenizer, token_sequences


def tokens_to_words(tokenizer, seq):
    word_seq = []
    for index in seq:
        if index == PAD_INDEX:
            word_seq.append("PAD")
        elif index == OOV_INDEX:
            word_seq.append(OOV_WORD)
        elif index == STOP_INDEX:
            word_seq.append('START')
        elif index == STOP_INDEX:
            word_seq.append('STOP')
        else:
            word_seq.append(tokenizer.sequences_to_texts([[index]])[0])
    print(word_seq)

In [9]:
src_seq, dest_seq = read_file_combined(FILE_NAME, MAX_LENGTH)

src_tokenizer, src_token_seq = tokenize(src_seq)
dest_tokenizer, dest_token_seq = tokenize(dest_seq)

In [10]:
dest_target_token_seq = [x + [STOP_INDEX] for x in dest_token_seq]  # 目标后加上 STOP
dest_input_token_seq = [[STOP_INDEX] + x for x in dest_target_token_seq]

src_input_data = sequence.pad_sequences(src_token_seq)
dest_input_data = sequence.pad_sequences(dest_input_token_seq, padding='post')

dest_target_data = sequence.pad_sequences(dest_target_token_seq, padding='post', maxlen=len(dest_input_data[0]))

In [29]:
rows = len(src_input_data[:, 0])
all_indices = list(range(rows))


<tf.Tensor: shape=(2, 2, 2), dtype=int32, numpy=
array([[[ 94, 100],
        [229, 244]],

       [[508, 532],
        [697, 730]]])>